In [1]:
import readline
import rpy2.robjects
import code.independence_test as it
import code.additive_noise as an
import numpy as np
import numpy.random as rn
import code.hsic as hs
import pandas as pd

In [2]:
n=500
uni = rn.uniform(0,10,n)
exp = rn.exponential(7,n)
gamma = rn.gamma(10,1,n)
log = rn.logistic(10,0.5,n)
n_Y = np.random.normal(0,0.4,n)
n_L = np.random.normal(0,0.5,n)
#Y = uni + exp + gamma + log + n_Y
Y = gamma + n_Y
leak = Y + n_L

In [3]:
X = np.stack([gamma, leak, log],axis=1)

In [4]:
X = pd.DataFrame(X, columns=['cause', 'leak', 'independent'])

In [ ]:
an.ANM_algorithm(X,Y, 'r')